In [ ]:
# There are several steps, many of them manual,
# in cleaning the MGP data.

# First, we load the tab-delimited files, if possible.

import matplotlib.pyplot as plt
import os
import pandas as pd
import re
from tqdm import tqdm_notebook as tqdm

In [ ]:
academic = pd.read_csv('./MGP_official/geneal_20190711/academic.tsv', sep='\t')

In [ ]:
advises  = pd.read_csv('./MGP_official/geneal_20190711/advises.tsv', sep='\t')

In [ ]:
country  = pd.read_csv('./MGP_official/geneal_20190711/country.tsv', sep='\t')

In [ ]:
degree_grant = pd.read_csv('./MGP_official/geneal_20190711/degree_grant.tsv', sep='\t')

In [ ]:
school   = pd.read_csv('./MGP_official/geneal_20190711/school.tsv', sep='\t')

In [ ]:
# These five tables load without a problem.
# The degree table, on the other hand, is riddled with issues.

#degree   = pd.read_csv('./MGP_official/geneal_20190711/degree.tsv', sep='\t')
degree   = pd.read_csv('./MGP_official/geneal_20190711/degree.csv')
#                        dtype={'degree_id': int, 'academic':  int, 'thesis':    str,
#                               'year':      int, 'msc':       int, 'degree_type': str })
# A note for the MGP: do not allow commas or tabs in the dissertation year field.

In [1]:
# The degree table required massive amounts 
# of manual cleaning to clear out excessive commas, quotes, 
# and other stray characters.

# In addition, there were several entries (many over 200 years old)
# that listed two dissertations and two degrees in one entry.
# These needed to be split, for purposes of counting individual degrees.

In [ ]:
for c in degree.columns:
    print(f"{c} has {degree[c].isna().sum()} NA.")

In [ ]:
# Begin cleaning the degree table.

#len(degree['msc'])  # 246,182
#degree['msc'].isna().sum()   # 107,998?! Nearly half.
degree['msc'] = degree['msc'].fillna(-1)
degree['msc'] = degree['msc'].astype('int64')
#degree['msc'].value_counts()

#degree['year'].isna().sum()
degree['year'] = degree['year'].fillna(-1)
degree['year'] = degree['year'].astype('int64')
#degree['year'].value_counts()

#degree['thesis'].isna().sum()
degree['thesis'] = degree['thesis'].fillna("")

#degree['degree_type'].isna().sum()
degree['degree_type'] = degree['degree_type'].fillna("")

In [ ]:
# I found it much easier to read the degree table in line-by-line
# to diagnose the issues.
with open('./MGP_official/geneal_20190711/degree.csv', 'r') as f:
    degree_text = f.readlines()

In [ ]:
regex_old_dual_degree = r"([0-9]+),([0-9]+),\"([^\"]+)(;|,)([^\"]+)\",([0-9]*),([0-9]*),([^,]+),([^,]+)\n"
find_old = re.compile(regex_old_dual_degree)
#replace_old_twoline = r"\1,\2,

for d in degree_text:
    if find_old.match(d):
        print(f"found in {degree_text.index(d)}: {d}")

In [ ]:
missing_quote_regex = r"([0-9]+),([0-9]+),[A-Z]"
missing_quote = re.compile(missing_quote_regex)

In [ ]:
num_commas, num_quotes = [], []
idx_commas, idx_quotes = dict(), dict()

for i in tqdm(range(len(degree_text))):
#for i in error_quotes:
    d = degree_text[i]
#    if d.count('"') not in [0,2]:
#        remove_thesis = r"(\"[A-Za-z0-9,.;]+\")"
#        d = re.sub(remove_thesis, "", d)
#        print(d)
    q = d.count('"')
    num_quotes.append(q)
    idx_quotes[q] = idx_quotes.get(q, []) + [i]
#    if i % 25000 == 0:
#        print(f"{i} completed.")
print("idx_quotes completed.")

# note: this should be tracking 
# *only* *delimiting* commas, 
# *not* *all* commas. ah well.
for i in tqdm(range(len(degree_text))):
#for i in error_commas:
    d = degree_text[i]
    c = d.count(',')
    num_commas.append(c)
    idx_commas[c] = idx_commas.get(c, []) + [i]
#    if i % 25000 == 0:
#        print(f"{i} completed.")
print("idx_commas completed.")


In [ ]:
# A common check on the entries in degree 
# were the number of commas and double quotes
# in the comma-delimited version of the file.
quotes, commas = [], []
for i in range(0,20):
    commas.append(num_commas.count(i))
    quotes.append(num_quotes.count(i))
print(f"commas: {commas}")
print(f"quotes: {quotes}")

In [ ]:
three_quotes_regex = r"([0-9]+),([0-9]+),\"([^\"]+)\"([^\"]*)\","
three_quotes_fix   = r'\1,\2,"\3\4",'
missing_quote = re.compile(missing_quote_regex)
#for i in idx_quotes[3]:
#    print(f"{i}: {degree_text[i]}")
#    fixed = re.sub(three_quotes_regex, three_quotes_fix, degree_text[i])
#    print(fixed)

In [ ]:
def count_commas_quotes_on_bad(entry):

    num_c = entry.count(',')
    num_q = entry.count('"')
    first_comma  = entry.index(',')
    second_comma = entry[first_comma +1:].index(',')
    third_comma  = entry[second_comma+1:].index(',')
    print(entry)
    print(third_comma)
    # for each entry with 0 quotes,
    # check if there are any alphabetic characters between commas 2 and 3.
#    pass_first_two_fields_regex = r"([0-9]+),([0-9]+),"
    if num_q == 0 and third_comma == second_comma + 1:
        return "no title"
    # for each entry off 5 commas and >2 quotes, 
    # count the commas and quotes INSIDE the main quotes.


In [ ]:
all_indices  = list(range(len(degree_text)))
error_quotes = sorted(list(set(all_indices) - set(idx_quotes[2])))
error_commas = sorted(list(set(all_indices) - set(idx_commas[5])))
# USE THESE FROM HERE ON TO RESTRICT THE INDICES OF THE ERROR LINES!

In [ ]:
def number_of_pairs(list1, list2):
    # return the number of entries in list1 
    # that are immediately before an entry in list2.
    total = 0
    for x in list1:
        for y in list2:
            if y - x == 1:
                total += 1
                break
    return total

In [ ]:
num_no_MSC = []
for i in range(len(degree_text)):
    if ",,Ph.D." in degree_text[i] and degree_text[i].count('"') >= 2:
        num_no_MSC.append(i)
len(num_no_MSC) # this is a lower bound; only the PhDs without MSC

In [ ]:
# let's find out the number of people each of these schools hosted
# to see how much it matters that I have these schools or not

degrees_granted = []  # will hold pairs: (school_id, # of degrees granted)
for s in list(school['school_id']):
    degrees_granted.append((s, len(degree_grant[degree_grant['school']==s])))

In [ ]:
degrees_granted = sorted(degrees_granted, key=lambda x: x[1], reverse=True)

In [ ]:
for s in degrees_granted:
    name = list(school[school['school_id']==s[0]]['school_name'])[0] # really?
    print(f"{name} has produced {s[1]} degrees listed in the MGP.")

In [ ]:
total_unlisted = 0
for s in degrees_granted:
    name = list(school[school['school_id']==s[0]]['school_name'])[0] # really?
    if s[0] in school_old_name_ids:
        print(f"{name} has produced {s[1]} degrees listed in the MGP.")
        total_unlisted += s[1]
print(f"Total degrees listed here: {total_unlisted}")  
# 696

In [ ]:
# many other cleaning processes, automatic and manual, not documented here.